# Multi-Agent AI Campaign Planning System on AWS

## Introduction
This notebook deploys a multi-agent AI campaign planning system using AWS services (Bedrock, SageMaker, Lambda, Step Functions, etc.), fully deploys and configures all required AWS resources, and supports both Gen-Z and enterprise use cases. Realistic dummy data and templates for consumer and business audiences are included.

## 1. Setup AWS Environment

In [ ]:

# Set AWS credentials and region securely
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'YOUR_AWS_ACCESS_KEY'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'YOUR_AWS_SECRET_KEY'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'  # Ensure Bedrock support


In [ ]:

# Install required libraries
!pip install boto3


## 2. Provision AWS Resources

### 2.1 Create S3 Buckets

In [ ]:

import boto3, json, random, string

region = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
s3 = boto3.client('s3', region_name=region)

suffix = ''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(6))
briefs_bucket = f"ai-campaign-briefs-{suffix}"
assets_bucket = f"ai-campaign-assets-{suffix}"

for bucket in [briefs_bucket, assets_bucket]:
    s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={'LocationConstraint': region})
    print(f"Created bucket: {bucket}")


### 2.2 Create DynamoDB Table

In [ ]:

dynamodb = boto3.client('dynamodb', region_name=region)

table_name = f"AI_CampaignData_{suffix}"
table_resp = dynamodb.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {"AttributeName": "campaign_id", "AttributeType": "S"},
        {"AttributeName": "step", "AttributeType": "S"}
    ],
    KeySchema=[
        {"AttributeName": "campaign_id", "KeyType": "HASH"},
        {"AttributeName": "step", "KeyType": "RANGE"}
    ],
    BillingMode='PAY_PER_REQUEST'
)
dynamodb.get_waiter('table_exists').wait(TableName=table_name)
print(f"DynamoDB Table created: {table_name}")


### 2.3 Populate Dummy Brief to S3

In [ ]:

campaign_brief = {
    "campaign_id": "CAMPAIGN123",
    "campaign_name": "GenZ Social Hype 2025",
    "target_audience": "Gen-Z",
    "product": "Tech Gadget X",
    "objectives": "Drive brand engagement among Gen-Z.",
    "key_messages": "Innovative, trendy, community-driven.",
    "channels": "social_media",
    "timeline": "Q3 2025",
    "budget": "50k USD"
}

brief_key = "sample_brief_genz.json"
s3.put_object(Bucket=briefs_bucket, Key=brief_key, Body=json.dumps(campaign_brief))
print(f"Uploaded brief to s3://{briefs_bucket}/{brief_key}")


## 3. Validate Setup
Check S3 buckets and DynamoDB table creation.

In [ ]:

print("Buckets:", [bucket['Name'] for bucket in s3.list_buckets()['Buckets']])
print("DynamoDB Tables:", dynamodb.list_tables()['TableNames'])


## Next Steps

Continue by creating Lambda functions, Step Functions workflow, and setting up Kendra and SNS according to the provided comprehensive notebook content. Ensure proper IAM permissions are configured.

Replace placeholders (`YOUR_AWS_ACCESS_KEY`, `YOUR_AWS_SECRET_KEY`) securely, ideally using IAM roles.